In [19]:
import pandas as pd
import numpy as np
import random, os, sys 
from groo.groo import get_root

os.path.join(get_root(".tasks_root"))

'/data/drive/postdoc/grants/resources/tasks-for-grants'

In [20]:


def gen_bin_arr(size, no_relevant):

    if no_relevant > size:
        raise ValueError("Number of relevant values (1s) cannot exceed the size of the array.")
    
    # Step 1: Generate the binary array
    binary_array = np.zeros(size, dtype=int)
    binary_array[:no_relevant] = 1
    np.random.shuffle(binary_array)
    
    # Step 2: Assign sequential numbers to positions where the binary array has 1s
    sequential_numbers = np.arange(1, no_relevant + 1)  # Generate sequential numbers starting from 1
    np.random.shuffle(sequential_numbers)  # Shuffle the sequential numbers
    result_array = np.zeros_like(binary_array)  # Create an array of the same shape as the binary array
    result_array[binary_array == 1] = sequential_numbers  # Assign shuffled numbers to 1 positions
    
    return binary_array, result_array



In [21]:
no_relevant = 3
no_contexts = 6
no_shown_t = 3

reps = 1
n_targets = 2

noise = 2 # standard deviation 

n_stim_total = 10

conds = ["low", "low-mid", "mid", "mid-high", "high"]
lvls = dict({
    "irr": [0, 0],
    "low": [5, -15],
    "low-mid": [15, -5],
    "mid": [20, -20], 
    "mid-high": [20, -40],
    "high": [-40, 20]
})

# trial no, target, c1_id, c1_v, c2_id, c2_v, c3_id, c3_v


basic_sched = pd.DataFrame()
ii = 1
for no_relevant in [3]:
    random_keys = random.sample(list(conds), no_relevant)
    ordered_keys = sorted(random_keys, key=lambda k: lvls[k][1], reverse=True)  # Sort by the upper bound descending

    while len(ordered_keys) < no_contexts:
        ordered_keys.append("irr")
    relevance = ["rel" if key != "irr" else "irr" for key in ordered_keys]

    stimuli = list(range(1, n_stim_total + 1))  # Stimuli numbered from 1 to 10
    stim_chosen = no_relevant*10 + np.array(random.sample(stimuli, no_contexts))

    positions = list(range(1, 7))  # Stimuli numbered from 1 to 10
    stim_positions = random.sample(positions, no_contexts)


    ## each rep will contain 2 trials (each for one target)
    for r in range(reps):
        print(r)
        for t in range(n_targets):
            #for s in range()
            shown, order = gen_bin_arr(no_contexts,no_shown_t) 
            tdf = pd.DataFrame({"tr_id": ii,
                                "no_rel_ctxts": no_relevant,
                                "target": t+1, 
                                "rel_cond": "rel"+str(no_relevant),
                                "relevance": relevance, 
                                "condition": ordered_keys,  
                                "condition_id": np.arange(1,no_contexts+1),
                                "outcome": [lvls[x][t] for x in ordered_keys],
                                "stim": stim_chosen,  
                                "stim_positions": stim_positions,
                                "shown": shown, 
                                "order": order
                                })
            
            basic_sched = pd.concat([basic_sched, tdf])
            ii = ii + 1
        


df2 = basic_sched.copy()
#basic_sched.to_csv(os.path.join(get_root(".tasks_root"), "contextual-inference", "schedules", "sch1_filtered.csv"))


# filter and re-order schedule
basic_sched = basic_sched.loc[basic_sched["shown"]==1,]

basic_sched = basic_sched.set_index(["tr_id", "order"]).unstack("order")

# Flatten column multi-index

basic_sched.columns = [
    f"{col[0]}_stim{col[1]}" if col[1] else col[0] for col in basic_sched.columns
]
constant_columns = [ "no_rel_ctxts", "target", "rel_cond", "target"]

df_constants = df2[["tr_id"] + constant_columns].drop_duplicates().set_index("tr_id")

# list of columns which need to be separated 
per_sim = ["relevance", "condition", "condition_id", "outcome", "stim", "stim_positions"]
basic_sched = df_constants.merge(basic_sched.loc[:,[v + "_stim"+ str(st) for v in per_sim for st in [1,2,3]] ], left_index=True, right_on="tr_id")

# calculate total outcome
basic_sched["outcome"] = basic_sched.filter(regex="outcome").sum(axis=1)

basic_sched.to_csv(os.path.join(get_root(".tasks_root"), "contextual-inference", "schedules", "sch1.csv"))
df2.to_csv(os.path.join(get_root(".tasks_root"), "contextual-inference", "schedules", "sch1_full.csv"))


0
